In [ ]:
#importing everything required for the lab
import numpy as np
import time
from tensorflow import keras
from tqdm import tqdm
from sklearn import metrics

# Copying functions from previous labs

## Convolution

In [ ]:
# forward propagation for convolution
def conv2d_forward(matrix, filter):
  h_x, w_x, _ = matrix.shape # getting matrix shape
  h_w, w_w, _  = filter.shape # getting filter shape
  output = np.zeros((h_x - h_w + 1, w_x - w_w + 1)) # initializing output matrix
  for i in range(len(output)): # for each pixel
    for j in range(len(output[i])):
        output[i][j] = np.sum(matrix[i:i+h_w, j:j+w_w, :] * filter) # calculate sum of hadamart product between
                                                                    # matrix batch and filter, save value in output cell
  return output

In [ ]:
# backward propagation for convolution (dL/dZ)
def conv2d_backward(upstream, filter):
  h_w, w_w, d_w  = filter.shape # getting filter shape
  padded_upstream = np.pad(upstream,((h_w - 1, h_w - 1), (w_w - 1, w_w - 1), (0, 0)), 'constant') # padding upstream matrix
  rotated_filter = np.rot90(np.rot90(filter)) # rotate filter by 180 degree
  dL_dZ = [] # initializing output
  for i in range(d_w): # for each channel
    dL_dZ.append(conv2d_forward(padded_upstream, rotated_filter[:, :, i, np.newaxis])) # adding dL/dZ
  return np.array(dL_dZ)

In [ ]:
# backward propagation for convolution (dL/dW)
def conv2d_backward_weights(weights, upstream):
  h_x, w_x, d_x  = weights.shape # getting filter shape
  dL_dZ = [] # initializing output
  for i in range(d_x): # for each channel
    dL_dZ.append(conv2d_forward(weights[:, :, i, np.newaxis], upstream)) # adding dL/dZ
  return np.transpose(np.array(dL_dZ), (1, 2, 0))

## ReLU

In [ ]:
a = np.array([[-0.00590765,  0.18932873],
       [-0.32396051,  0.25586596],
       [ 0.22358098,  0.02217555]])
1 * (a > 0)

array([[0, 1],
       [0, 1],
       [1, 1]])

In [ ]:
#function for RelU forward and backward propagation
#taken from previous assignments
def RelU_jacobian(input):
  return 1 * (input > 0)

def RelU_forward_prop(input):
  return np.maximum(input, 0)

def RelU_backward_prop(input, loss):
  jac = RelU_jacobian(input) # finding jacobian for RelU according to input
  return jac * np.array(loss)

## Matmul

In [ ]:
#functions for matmul backward and forward propagation from previous assignments
def MatMul_forward_prop(matrix, input):
  return np.array(matrix) @ np.array(input)

#function that finds dL/dx
def MatMul_backward_prop(matrix, loss):
  return np.array(matrix).T @ np.array(loss)

#function that finds dL/dW
def MatMul_matrix_backward_prop(X, loss):
  return np.array(loss) @ np.array(X).T

## SoftMax

In [ ]:
#function for softmax forward and backward propagation
#taken from previous assignments
def SoftMax_forward_prop(input, normalization=False):
  output = np.array(input, dtype=np.longdouble)
  if normalization: # if we use normalization
    output = output - np.max(input) # we substract maximal value from each number
  output = np.exp(output)
  return output / np.sum(output)

def SoftMax_jacobian(input, normalization=False): # function for calculating jacobian of SoftMax according to input
  output = SoftMax_forward_prop(input, normalization)
  jacobian = np.zeros((len(input), len(input)))
  for i in range(len(input)):
    for j in range(len(input)):
      if i == j:
        jacobian[i][j] = output[i] * (1 - output[j])
      else:
        jacobian[i][j] = -output[i] * output[j]
  return jacobian

def SoftMax_backward_prop(input, loss, normalization=False): # backpropagation
  jac = SoftMax_jacobian(input, normalization) # calculating jacobian
  return jac @ np.array(loss)

## Log softmax

In [ ]:
#functions for log_softmax forward and backward propagation
#this node applies softmax and then finds logorithm of the result
def log_softmax(x):
  x_max = np.max(x)
  return x - x_max - np.log(np.sum(np.exp(x - x_max)))

def log_softmax_jacobian(input): # function for calculating jacobian of SoftMax according to input
  output = SoftMax_forward_prop(input, True)
  jacobian = np.zeros((len(input), len(input)))
  for i in range(len(input)):
    for j in range(len(input)):
      if i == j:
        jacobian[i][j] = (1 - output[j])
      else:
        jacobian[i][j] = -output[j]
  return jacobian

def log_softmax_backward_prop(input, loss): # backpropagation
  jac = log_softmax_jacobian(input) # calculating jacobian
  return jac @ np.array(loss)

## Labels vectorization

In [ ]:
#function taken from previous assignments
#it translates label number into the vector of 0s and 1s
def label_vec_func(labels):
  labels_matrix = np.zeros([len(labels), c])
  for i in range(len(labels)):
    labels_matrix[i, labels[i]] = 1
  return labels_matrix

# Implementing functions required for this lab

In [ ]:
# forward propagation for convolution
# now we have filters - array of size: (height, width, depth, number of filters)
def conv2d_forward_many(matrix, filters):
  h_x, w_x, _ = matrix.shape # getting matrix shape
  h_w, w_w, _, d  = filters.shape # getting filter shape
  output = np.zeros((h_x - h_w + 1, w_x - w_w + 1, d)) # initializing output matrix
  for k in range(d): #for each filter
    output[:, :, k] = conv2d_forward(matrix, filters[:, :, :, k])
  return output

In [ ]:
# backward propagation for convolution (dL/dZ)
# now we have filters - array of size: (height, width, depth, number of filters)
def conv2d_backward_many(upstream, filters):
  h_w, w_w, d_w, D  = filters.shape # getting filter shape
  dL_dZ = []
  for i in range(D): # for each filter
    dL_dZ.append(conv2d_backward(upstream[:, :, i, np.newaxis], filters[:, :, : , i]))
  return np.transpose(np.sum(dL_dZ, 0), (1, 2, 0)) # transpose, since filters size is first after np.array

In [ ]:
# backward propagation for convolution (dL/dW)
# now we have filters - array of size: (height, width, depth, number of filters)
def conv2d_backward_weights_many(weight, upstream):
  _, _, D  = upstream.shape # getting filter shape
  dL_dWs = [] # initializing output
  for i in range(D): # for each channel
    dL_dWs.append(conv2d_backward_weights(weight, upstream[:, :, i, np.newaxis]))
  return np.transpose(np.array(dL_dWs), (1, 2, 3, 0)) # transpose, since filters size is first after np.array

## Testing new function

### Forward propagation

In [ ]:
# initializing input array and filter
arr1d = np.array([[[1], [2]], [[3], [4]]])

filt1d = np.array([[[[0.79559247, 0.520825  , 0.99224229]],

        [[0.08872951, 0.47740497, 0.69036656]]]])

In [ ]:
# calculating convolution forward propagation using my function
print(conv2d_forward_many(arr1d, filt1d))

[[[0.97305149 1.47563494 2.37297541]]

 [[2.74169545 3.47209488 5.73819311]]]


In [ ]:
import torch
from torch.nn.functional import conv2d

In [ ]:
# initializing input array and filter as torch tensors
custom_filter = torch.tensor(filt1d.transpose(3,2,0,1), dtype=float, requires_grad=True)
input_data = torch.tensor(arr1d, dtype=float, requires_grad=True)
input_matrix = input_data.unsqueeze(3).permute(2, 3, 0, 1)

In [ ]:
# calculating convolution forward propagation using pytroch
output = conv2d(input_matrix, custom_filter, padding=0)
print(output.permute(0, 3, 2, 1).detach().numpy())

[[[[0.97305149 1.47563494 2.37297541]
   [2.74169545 3.47209488 5.73819311]]]]


### Backprop

$\frac{∂\bf L}{∂\bf Z_{k-1}}$

In [ ]:
# initializing input array and loss as torch tensors
loss = torch.randn(1, 3, 3, 1, dtype=float,  requires_grad=True)


input_matrix = torch.randn(1, 1, 3, 2, dtype=float,  requires_grad=True) # dL/dZ does not depend on input matrix according to lecture slides

In [ ]:
input_matrix.shape

torch.Size([1, 1, 3, 2])

In [ ]:
# calculating convolution forward propagation using my function
out = conv2d_forward_many(input_matrix.permute(0,2,3,1).squeeze(0).detach().numpy(), filt1d)
print(out)

[[[-0.52044866 -1.17758066 -1.80606315]]

 [[-0.42567757 -0.64739822 -1.04066509]]

 [[-0.72806595 -1.09877945 -1.76815584]]]


In [ ]:
out.shape

(3, 1, 3)

In [ ]:
# calculating convolution backward propagation using pytroch
output = conv2d(input_matrix, custom_filter)
print(output.permute(2, 3, 0, 1))

tensor([[[[-0.5204, -1.1776, -1.8061]]],


        [[[-0.4257, -0.6474, -1.0407]]],


        [[[-0.7281, -1.0988, -1.7682]]]], dtype=torch.float64,
       grad_fn=<PermuteBackward0>)


In [ ]:
loss.permute(2, 3, 0, 1).shape

torch.Size([3, 1, 1, 3])

In [ ]:
output.backward(loss)
input_matrix.grad

tensor([[[[-1.8941, -1.3050],
          [ 0.7315,  0.3237],
          [-1.4082, -0.8771]]]], dtype=torch.float64)

In [ ]:
loss.permute(0, 2, 3, 1).squeeze(0).detach().numpy().shape

(3, 1, 3)

In [ ]:
# calculating convolution forward propagation using my function
print(conv2d_backward_many(loss.permute(0, 2, 3, 1).squeeze(0).detach().numpy(), filt1d))

[[[-1.89406093]
  [-1.30497249]]

 [[ 0.73148394]
  [ 0.32373761]]

 [[-1.40818568]
  [-0.87711921]]]


#### $\frac{∂\bf L}{∂\bf W}$

In [ ]:
custom_filter.grad.permute(2,3,1,0).detach().cpu().numpy()

array([[[[0.20162918, 0.80112189, 0.98341681]],

        [[0.70193302, 2.70520594, 3.28438387]]]])

In [ ]:
input_matrix = input_matrix.detach().squeeze(0).permute(1, 2, 0).numpy()
input_matrix = input_matrix

In [ ]:
loss = loss.permute(0, 2, 3, 1).squeeze(0).detach().numpy()

In [ ]:
# calculating convolution forward propagation using my function
a = conv2d_backward_weights_many(input_matrix, loss)
print(a)

[[[[0.20162918 0.80112189 0.98341681]]

  [[0.70193302 2.70520594 3.28438387]]]]


# Downloading and preprocessing dataset

In [ ]:
# loading cifar10 dataset
dataset=keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = dataset.load_data() #loading data
c = 5 # number of labels

Let's decrease samples in dataset in order to speed up the training process

In [ ]:
def decrease_mnist_samples(data, new_num_classes, max_repeat):
  # decreasing dataset by number of classes and number of samples for each class
  classes = range(new_num_classes) # classes that stayed
  repeations = {i:0 for i in classes} # dictionary for repetitions
  decreased_X, decreased_Y = [], [] # new X and Y
  for x, y in data:
    if y in classes: # for remained class
      if repeations[y] < max_repeat: # check number of repetitions
        repeations[y]+=1 # update this number
        decreased_X.append(x) # add new value to X
        decreased_Y.append(y) # add new value to Y
  # shuffle new dataset
  np.random.shuffle(decreased_X)
  np.random.shuffle(decreased_Y)
  return np.array(decreased_X), np.array(decreased_Y)

In [ ]:
(x_train, y_train)= decrease_mnist_samples(zip(x_train, y_train), c, 160)
(x_test, y_test) = decrease_mnist_samples(zip(x_test, y_test), c, 20)
y_train = label_vec_func(y_train) #converting labels to vector of 0s and 1s
N_train = y_train.shape[0] #number of pictures in the train dataset
N_test = y_test.shape[0] #number of pictures in the test dataset

In [ ]:
x_train.shape

(800, 28, 28)

In [ ]:
h, w = 28, 28
n = h * w #number of pixels for one picture

In [ ]:
hw1, Ww1, d, D1 = 1, 1, 1, 2 # initialize kernel sizes, depth and number of filters for first layer

hw2, Ww2, D2 = 1, 1, 2 # initialize kernel sizes, number of filters for second layer

Conv1 =  np.random.uniform(-1, 1, (hw1, Ww1, d, D1)) # initial convolution filters for layer 1
b1 = np.random.uniform(-1, 1 , (h - hw1 + 1, h - Ww1 + 1, D1)) #b1 - initial bias
Conv2 =  np.random.uniform(-1, 1, (hw2, Ww2, D1, D2)) # initial convolution filters for layer 2
b2 = np.random.uniform(-1, 1 , (h - hw1 - hw2 + 2, h - Ww1 - Ww2 + 2, D2)) #b2 - initial bias

vectorized_len = (h - hw1 - hw2 + 2 ) * (h - Ww1 - Ww2 + 2) * D2 # size for vectorized tensor after 2 convulitions

c1 = vectorized_len // 4 # dimension of the first layer
W1 = np.random.uniform(-1, 1, (c1, vectorized_len)) #W1 - initial weights
W2 = np.random.uniform(-1, 1, (c, c1)) #W2 - initial weights
b3 = np.random.uniform(-1, 1 , (c1, 1)) #b1 - initial bias
b4 = np.random.uniform(-1, 1 , (c, 1)) #b2 - initial bias


nu = 0.005 # learning rate
num_epochs = 10 # amount of epochs

N = 2 # number of images in minibatch

# initialize partial derivatives
dL_dConv1 = np.zeros((h - hw1 + 1, h - Ww1 + 1, D1))
dL_dConv2 = np.zeros((hw1, Ww1, d, D1))
dL_dW2 = np.zeros((c1, vectorized_len))
dL_dW2 = np.zeros((c, c1))
dL_db1 = 0
dL_db2 = 0
dL_db3 = 0
dL_db4 = 0

In [ ]:
for i in range(num_epochs): #for each epoch
  total_loss = 0 #sum of losses for one epoch
  correct = 0 #number of correct predictions
  counter = 0 #counter to check that batch ended
  for i in tqdm(range(N_train)): #for each picture

    x = x_train[i, :, : , np.newaxis] / 255 #normalize pixels, so they will be from 0 to 1

    y_true = np.array(y_train[i].reshape(c,  1))  #true value of y

    #forward propagation

    y1 = conv2d_forward_many(x, Conv1) #applying convoltion filters from layer 1
    y2 = y1 + b1 #adding bias
    y3 = RelU_forward_prop(y2) #applying RelU
    y4 =  conv2d_forward_many(y3, Conv2) #applying convoltion filters from layer 2
    y5 = y4 + b2 #adding bias
    y6 = RelU_forward_prop(y5) #applying RelU

    y7 = np.reshape(y6, (vectorized_len, 1))

    y8 = MatMul_forward_prop(W1, y7) #applying matrix multiplication with the first weight matrix
    y9 = y8 + b3 #adding bias
    y10 = RelU_forward_prop(y9) #applying RelU
    y11 =  MatMul_forward_prop(W2, y10) #applying matrix multiplication with the second weight matrix
    y12 = y11 + b4 #adding bias
    y13 = log_softmax(y12) #applying log softmax

    if np.argmax(y13) == np.argmax(y_true): #check if predictions are correct
      correct += 1

    loss = y_true.T @ y13 #finding loss
    total_loss += loss #adding current loss to total loss

    #backward propagation

    back = -y_true + SoftMax_forward_prop(y12, True) #backpropagation from loss to the input of softmax

    dL_db4 += back #backpropagation from loss to the input of addition, finding dL/db4

    dL_dW2 += MatMul_matrix_backward_prop(y10, back) #fiding dL/dW2

    back = MatMul_backward_prop(W2, back) #backpropagation from loss to the input of matrix multiplication with matrix W2

    back = RelU_backward_prop(y9, back) #backpropagation from loss to the input of RelU

    dL_db3 = back #backpropagation from loss to the input of addition, finding dL/db1

    dL_dW1 = MatMul_matrix_backward_prop(y7, back) #fiding dL/dW1

    back = MatMul_backward_prop(W1, back) #backpropagation from addition to the input of matrix multiplication with matrix W1

    back = back.reshape((h - hw1 - hw2 + 2, h - Ww1 - Ww2 + 2, D2)) # backpropagation of reshaping

    back = RelU_backward_prop(y5, back) # backpropagation from reshaping to the input of RelU

    dL_db2 = back #backpropagation from ReLU to the input of addition, finding dL/db2

    dL_dConv2 = conv2d_backward_weights_many(y3, back) #finding dL/dConv2

    back = conv2d_backward_many(back, Conv2) # backpropagation from addition to the input convolution
    back = RelU_backward_prop(y2, back) # backpropagation from convolution to the input of RelU

    dL_db1 = back #backpropagation from ReLU to the input of addition, finding dL/db1

    dL_dConv1 = conv2d_backward_weights_many(x, back) #finding dL/dConv1

    counter += 1  # increasing counter

    if counter == N or i == N_train - 1: # if batch ended or dataset ended. We apply gradient descent only in batches
    #applying gradient descent for weights and biases

      Conv1 = Conv1 - nu * dL_dConv1
      Conv2 = Conv2 - nu * dL_dConv2
      W1 = W1 - nu / N * dL_dW1
      W2 = W2 - nu / N * dL_dW2
      b1 = b1 - nu / N * dL_db1
      b2 = b2 - nu / N * dL_db2
      b3 = b3 - nu / N * dL_db3
      b4 = b4 - nu / N * dL_db4

      dL_dConv1 = np.zeros((h - hw1 + 1, h - Ww1 + 1, D1))
      dL_dConv2 = np.zeros((hw1, Ww1, d, D1))
      dL_dW2 = np.zeros((c1, vectorized_len))
      dL_dW2 = np.zeros((c, c1))
      dL_db1 = 0
      dL_db2 = 0
      dL_db3 = 0
      dL_db4 = 0


  print('\nAccuracy:', correct / N_train)
  print('Loss:', -total_loss.item() / N_train)


100%|██████████| 800/800 [01:02<00:00, 12.89it/s]



Accuracy: 0.49625
Loss: 269.44150649173284698


100%|██████████| 800/800 [00:58<00:00, 13.72it/s]



Accuracy: 0.51625
Loss: 102.33412987727251005


100%|██████████| 800/800 [00:58<00:00, 13.78it/s]


...


Accuracy: 0.525
Loss: 60.108655925068598665


100%|██████████| 800/800 [01:06<00:00, 12.07it/s]



Accuracy: 0.50125
Loss: 92.106853230630311054


100%|██████████| 800/800 [00:59<00:00, 13.45it/s]


Accuracy: 0.5125
Loss: 106.409671040141248735


## Evaluation, taken from lab 6

In [ ]:
y_pred = np.zeros((N_test, 1), int) #predictions
for i in tqdm(range(N_test)):
    x = x_test[i, :, : , np.newaxis] / 255 #normalize pixels, so they will be from 0 to 1

    y1 = conv2d_forward_many(x, Conv1) #applying matrix multiplication with the first weight matrix
    y2 = y1 + b1 #adding bias
    y3 = RelU_forward_prop(y2) #applying RelU
    y4 =  conv2d_forward_many(y3, Conv2) #applying matrix multiplication with the second weight matrix
    y5 = y4 + b2 #adding bias
    y6 = RelU_forward_prop(y5) #applying RelU

    y7 = np.reshape(y6, (vectorized_len, 1))

    y8 = MatMul_forward_prop(W1, y7) #applying matrix multiplication with the first weight matrix
    y9 = y8 + b3 #adding bias
    y10 = RelU_forward_prop(y9) #applying RelU
    y11 =  MatMul_forward_prop(W2, y10) #applying matrix multiplication with the second weight matrix
    y12 = y11 + b4 #adding bias
    out = SoftMax_forward_prop(12, True) #applying softmax to find outputs

    y_pred[i] = np.argmax(out) #setting prediction (the greatest number among outputs)

100%|██████████| 100/100 [00:02<00:00, 36.33it/s]


In [ ]:
print("Accuracy using L1 distance:", metrics.accuracy_score(y_true=y_test, y_pred=y_pred))

Accuracy using L1 distance: 0.5


# Torch based model

In [ ]:
import torch.nn as nn
import torch

class ConvolutionalNN(nn.Module):
  def __init__(self):
    super().__init__()
    self.sequence = nn.Sequential(
        nn.Conv2d(d, D1, (hw1, Ww1), padding=0, bias=True), # Convolution layer 1 with the same parameters and with bias
        nn.ReLU(), # ReLU
        nn.Conv2d(D1, D2, (hw2, Ww2), padding=0, bias=True), # Convolution layer 2 with the same parameters
        nn.ReLU(), # RelU
        nn.Flatten(), # Vectorization
        nn.Linear(vectorized_len, c1), # Linear layer 1 with the same parameters, has bias by default
        nn.ReLU(),
        nn.Linear(c1, c), # Linear layer 2 with the same parameters, has bias by default
        nn.Softmax(dim=1) # Softmax
    )

  def forward(self, x):
        return self.sequence(x)

In [ ]:
model = ConvolutionalNN() # initializing model
optimizer = torch.optim.SGD(model.parameters(), lr=nu) # gradient descent with same learning rate
loss_fn = nn.CrossEntropyLoss() # same loss

In [ ]:
model.train()
for i in range(num_epochs): #for each epoch
  total_loss = 0 #sum of losses for one epoch
  X_batch = []
  Y_batch = []
  for i in tqdm(range(N_train)): #for each picture

    x = torch.tensor(x_train[i, np.newaxis, :, :] / 255, dtype=torch.float32)
    y = torch.tensor(y_train[i], dtype=torch.float32)

    if len(X_batch) > N or i == N_train - 1: # if batch ended
      # convert list of samples to tensors
      X = torch.stack(X_batch)
      Y = torch.stack(Y_batch)

      optimizer.zero_grad()
      output = model(X) # find output
      loss = loss_fn(output, Y) # send to loss
      total_loss += loss.item()
      loss.backward()
      optimizer.step()

      # clear batches
      X_batch = []
      Y_batch = []

    else:
      # add to batch
      X_batch.append(x)
      Y_batch.append(y)

  print('Loss:', total_loss / N_train)

100%|██████████| 800/800 [00:00<00:00, 1114.17it/s]


Loss: 0.23270041916519404


100%|██████████| 800/800 [00:00<00:00, 1079.89it/s]


Loss: 0.23115042701363564


100%|██████████| 800/800 [00:01<00:00, 700.05it/s]


...

Loss: 0.23001396730542184


100%|██████████| 800/800 [00:01<00:00, 699.22it/s]


Loss: 0.229956674054265


100%|██████████| 800/800 [00:01<00:00, 721.15it/s]

Loss: 0.22992470040917395


In [ ]:
y_pred = np.zeros((N_test, 1), int) #predictions
model.eval()
with torch.no_grad():
    for i in range(N_test): # for each sample in test
        x = torch.tensor(x_test[i, :].reshape(1, 1, 28, 28), dtype=torch.float32) # convert to tensor
        y_pred[i] = torch.argmax(model(x)) # find output of a model

In [ ]:
print("Accuracy using L1 distance:", metrics.accuracy_score(y_true=y_test, y_pred=y_pred))

Accuracy using L1 distance: 0.55
